In [ ]:
##The following code is developed to pick P- and S-wave arrivals on each station,
##for each event, placing these picks in an .obs file for NonLinLoc.

##This is not a professional picker, but has rather been made by Kasper van Wijk and
#myself (Meegan Soulsby) as an educational piece to understand this process.

#Import packages required
import matplotlib
import matplotlib.pyplot as plt
from obspy.core import UTCDateTime
from obspy.clients.fdsn import Client
from matplotlib.dates import num2date
from obspy.core.event import Catalog, Event, Pick
from obspy.core.event.base import WaveformStreamID

#backend that works on my Macbook 
matplotlib.use("TKAgg")


In [ ]:
#Setting up the interactive selection of P- and S-wave arrival time picks

def onclick(event):
    ''' Record the times of the user's DOUBLE clicks on P- and S-wave arrivals in the 
    global variable retval and close the figure '''
    if event.dblclick:
        global retval
        # save your picks to the retval list in UTCDateTime format. Note:
        # there must be a more elegant way than this:
        retval.append(UTCDateTime(num2date(event.xdata).isoformat()))
        # close the panel when you picked a P- and an S-wave arrival:
        if len(retval) == 2:
            plt.close()


def makeapick(st_sttn,retval,phase):
    '''Make a pick for the event. 
    INPUTS are the stream, the picks from clicks on the plot of the traces, and the phase of the wave (P or S).
    OUTPUT is the Pick() object pick.
    The following assumes that the first trace/pick is P and
    the second is S wave. '''
    pick = Pick()
    pick.phase_hint = phase
    if phase == 'P':
        seed_string = st_sttn[0].get_id()
        pick.waveform_id = WaveformStreamID(seed_string=seed_string)
        pick.time = retval[0]
    else:
        seed_string = st_sttn[1].get_id()
        pick.waveform_id = WaveformStreamID(seed_string=seed_string)
        pick.time = retval[1]
    return pick

In [ ]:
##Defining parameters used for the Auckland Volcanic Field's (AVF) seismicity. 

#The GeoNet database is used.
client= Client("GEONET")
net ='NZ'

#A string, converted to a list for the stations under consideration of hand-picked arrivals:
stn = 'ABAZ,AWAZ,EPAZ,ETAZ,GRZ,HBAZ,KBAZ,KUZ,MBAZ,MKAZ,MTAZ,RVAZ,TOZ,WIAZ,WTAZ'
stn_list = stn.split(',')

#Define the possible channels and locations of the stations of your catalogue:
chn ='HH*,EH*'
loc = '10,12'

#Define a start- and endtime to download within the Geonet catalogue
starttime=UTCDateTime("2012-01-01T00:00:01")
endtime = UTCDateTime.now()

#Define a box to look for events in the Geonet catalogue, and a minimum magnitude
[maxlat,minlat,minlon,maxlon,minmag] = [-36.5,-37.3,174.5,175.3,1.0]

#order your catalogue by descending orderby
orderby='magnitude'


In [ ]:
##Generate a catalogue of events. Note, there were 91 events at the time of this study

events = client.get_events(starttime=starttime, endtime=endtime, minlatitude=minlat, maxlatitude=maxlat, minlongitude=minlon, maxlongitude=maxlon, minmagnitude=minmag,orderby=orderby)
print(events)

#Define a new and empty catalog object: 
cat = Catalog()
cat.description = "A catalogue with AVF events from Geonet but with only hand-picked arrivals"


In [ ]:
##Picking on unfiltered waveform data

#Define the directory you wish to save your .obs files
writepath="/Users/meegansoulsby/documents/..."

#Loop through the events in the Geonet catalogue:
for event in events[0:2]: #Change this index in the square brackets. Can do all at once, or in stages.
    
    #Creating title for the output file as the GeoNet resource id
    title = str(event.resource_id).split('/')[1]

    #For every geonet event, create a new and empty event object to store our
    #manual picks:
    e = Event()
    e.event_type = event.event_type
   
    #Download waveforms for the event in the Geonet catalogue:
    t0 = event.origins[0].time
    st = client.get_waveforms(network=net,station=stn,location=loc,channel=chn,starttime=t0,endtime=t0+40) 
    
    #Plot the event on one station ( usually three channels) at a time:
    for sttn in stn_list:
        
        #Select the data stream per station:
        st_sttn = st.select(station=sttn)
        
        #Create an array for the new Pick objects:
        retval = [] #these are mouse clicks
        
        #Only continuing if there is a signal from this one station
        if len(st_sttn)>0:
            
            #Plotting data for all channels available for this station
            fig,axs = plt.subplots(figsize=(15,15))
            st_sttn.plot(fig=fig)
            cid = fig.canvas.mpl_connect('button_press_event', onclick)
            plt.show()
                
            #If no picks are made, move into the next station. Do this by closing the window.
            if not retval:
                print('no picks, moving on to next station.')
            
            #If there is only one pick to make, this code assumes it is the P-wave arrival
            elif len(retval)==1:
                p = makeapick(st_sttn,retval,'P')
                
                #add the P wave pick to list of picks in the pick attribute of event e
                e.picks = e.picks+[p]
            
            #If there P and S picks made, retval is 2 
            else:
                p = makeapick(st_sttn,retval,'P')
                s = makeapick(st_sttn,retval,'S')
                
                #Add the P and S wave picks to list of picks in the pick attribute of event e
                e.picks = e.picks+[p,s]
        
        #Append the event to the new catalogue, IF it has picks:
        if len(e.picks) > 0:
            cat.append(e)
            
            #Save .obs file for each event where you want them which was defined by writepath
            for event in cat:
                for pick in event.picks:
                    event.write(writepath+title+"_handpicks"+".obs", format="NLLOC_OBS")
                
